In [268]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página a scrapear
url = 'http://www.intecmar.gal/Informacion/biotoxinas/Evolucion/CierresBatea.aspx'

# Realizar la solicitud GET a la URL
response = requests.get(url)

# Verificar que la solicitud fue exitosa (código 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar la tabla por su ID (puedes inspeccionar el HTML para obtener el ID correcto)
    table = soup.find('table', {'id': 'ctl00_Contenido_GridView2'})
    
    if table:
        # Extraer los datos de la tabla y convertirlos a un DataFrame de pandas
        df = pd.read_html(str(table))[0]
        
        # Mostrar el DataFrame
        print(df)
        
        # También puedes guardar el DataFrame en un archivo CSV si lo deseas
        df.to_csv('diascierre_hoy_aux.csv', index=False)
    else:
        print('No se encontró la tabla en la página.')
else:
    print('No se pudo acceder a la página.')


         Polígono  Xan   Feb   Mar   Abr   Mai   Xun  Xull  Ago  Set  Out  \
0   Vilagarcía B1  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
1   Vilagarcía B2  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
2     A Pobra E.1  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
3     A Pobra E.2  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
4             NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
..            ...  ...   ...   ...   ...   ...   ...   ...  ...  ...  ...   
58    Redondela D  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
59    Redondela E  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
60         Vigo A  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
61            NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  NaN  NaN  NaN   
62       Baiona A  NaN  14.0  31.0  30.0  31.0  23.0   NaN  NaN  NaN  NaN   

    Nov  Dec  Total  
0   NaN  NaN    NaN  
1   NaN  NaN    NaN  
2   NaN  

C:\Users\rmarc\AppData\Local\Temp\ipykernel_9900\794804033.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [269]:
df_hoy = pd.read_csv('diascierre_hoy_aux.csv')
df_hoy = df_hoy.rename(columns={'Xan': '1', 'Feb': '2', 'Mar': '3', 'Abr': '4', 'Mai': '5', 'Xun': '6', 'Xull': '7', 'Ago': '8', 'Set': '9', 'Out': '10', 'Nov': '11', 'Dec': '12'})
#df_hoy[['Xan','Feb','Mar','Abr','Mai','Xun','Xull','Ago','Set','Out','Nov','Dec']] = df_hoy[['Xan','Feb','Mar','Abr','Mai','Xun','Xull','Ago','Set','Out','Nov','Dec']].astype(int)
df_hoy = df_hoy.fillna('0')

df_hoy = df_hoy[df_hoy['Polígono'] != '0']
df_hoy.to_csv('diascierre_hoy.csv', index=False)

In [270]:
from datetime import datetime, timedelta

hoy = datetime.now().date()
hoy_fecha = hoy.strftime('%Y-%m-%d')
ayer_fecha = hoy - timedelta(1)

In [271]:
df_hoy_cangasb = df_hoy[df_hoy['Polígono'] == 'Cangas B']
mes = hoy.month
mes = str(mes)
df_hoy_cangasb[mes]

40    8.0
Name: 6, dtype: object

In [272]:
df2 = pd.DataFrame()

for poligono in df_ayer['Polígono']:
    df2.insert(0,poligono,0, True)
df2.insert(0,'dia',0,True)
df2.to_csv('diascierreabiertocerradoscrap.csv', index=False)

In [273]:
df_ayer = pd.read_csv('diascierre_ayer.csv')

for index, row_ayer in df_ayer.iterrows():
    polig = row_ayer['Polígono']
    df_hoy_poligono = df_hoy[df_hoy['Polígono'] == polig]
    v_hoy = df_hoy_poligono[mes].iloc[0]
    df_ayer_poligono = df_ayer[df_ayer['Polígono'] == polig]
    v_ayer = df_ayer_poligono[mes].iloc[0]
    v_hoy = int(v_hoy)
    v_ayer = int(v_ayer)

    existe_valor = (df2['dia'] == ayer_fecha).any()

    if v_hoy != v_ayer:
        if existe_valor:
            df2[polig] = '1'
        else: 
            nueva_fila = {'dia' : ayer_fecha, polig : '1'}
            df2.loc[len(df2)] = nueva_fila
    else:
        if existe_valor:
            df2[polig] = '0'
        else: 
            nueva_fila = {'dia' : ayer_fecha, polig : '0'}
            df2.loc[len(df2)] = nueva_fila

df2.to_csv('prueba.csv', index=False)